In [1]:
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats
import xlsxwriter
from secrets import IEX_CLOUD_API_TOKEN
from statistics import mean

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


In [3]:
symbol = "AAPL"
api_url = f"https://sandbox.iexapis.com/stable/stock/{symbol}/stats/?token={IEX_CLOUD_API_TOKEN}"    
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2450496589330,
 'week52high': 163.13,
 'week52low': 109.4,
 'week52highSplitAdjustOnly': 158.65,
 'week52lowSplitAdjustOnly': 107.98,
 'week52change': 0.2342443484042541,
 'sharesOutstanding': 16544120820,
 'float': 0,
 'avg10Volume': 72733717,
 'avg30Volume': 82344444,
 'day200MovingAvg': 144.43,
 'day50MovingAvg': 149.6,
 'employees': 147075,
 'ttmEPS': 5.2,
 'ttmDividendRate': 0.888535085864821,
 'dividendYield': 0.005931616457425092,
 'nextDividendDate': '',
 'exDividendDate': '2021-07-30',
 'nextEarningsDate': '2021-10-15',
 'peRatio': 28.214410501280906,
 'beta': 1.3609491954696578,
 'maxChangePercent': 56.0254267427544,
 'year5ChangePercent': 4.279262067081001,
 'year2ChangePercent': 1.5059539495522278,
 'year1ChangePercent': 0.23222696398707904,
 'ytdChangePercent': 0.10001228422716787,
 'month6ChangePercent': 0.08677287904338793,
 'month3ChangePercent': -0.00945886618559885,
 'month1ChangePercent': -0.008503267156391107,
 'day30Change

In [4]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
symbol_groups = list(chunks(stocks['Ticker'], 100))
#print(symbol_groups)
symbol_strings = list()
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    print(symbol_strings[i])
    
my_columns = ["Ticker", "Price", "One-Year Price Return", "Number of Shares to Buy"]

A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA
CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD
GIS,GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,IEX,IFF,ILM

In [5]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series(
                [
                symbol,
                data[symbol]['quote']['latestPrice'],
                data[symbol]['stats']['year1ChangePercent'],
                'N/A'],
            index = my_columns),
        ignore_index=True)
final_dataframe


,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,157.27,0.452143,N/A
1,AAL,20.02,0.625353,N/A
2,AAP,231.28,0.419268,N/A
3,AAPL,149.12,0.231024,N/A
4,ABBV,109.78,0.330044,N/A
...,...,...,...,...
500,YUM,130.58,0.304583,N/A
501,ZBH,151.95,0.002307,N/A
502,ZBRA,542.34,0.793666,N/A
503,ZION,65.00,1.103382,N/A


In [6]:
final_dataframe.sort_values("One-Year Price Return", ascending=False, inplace=True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace=True)
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,148,DVN,42.27,3.640032,N/A
1,317,MRO,16.99,3.163737,N/A
2,175,FANG,114.46,2.896707,N/A
3,275,LB,82.96,2.319534,N/A
4,357,OXY,34.22,2.167293,N/A
5,299,MCHP,76.87,1.657786,N/A
6,410,SIVB,718.23,1.5712,N/A
7,23,ALB,246.21,1.522252,N/A
8,352,OKE,65.50,1.51269,N/A
9,160,EOG,90.95,1.457531,N/A


In [7]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the size of your portfolio: ")
    
    try:
        float(portfolio_size)
    except ValueError:
        print("THat is not a number! \nPlease try again: ")
        portfolio_size = input("Enter the size of your portfolio: ")

portfolio_input()

Enter the size of your portfolio: 1234567


In [8]:
position_size = float(portfolio_size)/len(final_dataframe.index)
print(position_size)

24691.34


In [9]:
for i in range(0, len(final_dataframe)):
    final_dataframe.loc[i, "Number of Shares to Buy"] = math.floor(position_size/final_dataframe.loc[i, "Price"])

final_dataframe

Z:\Projects\strategy_index_screener\.venv\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,148,DVN,42.27,3.640032,584
1,317,MRO,16.99,3.163737,1453
2,175,FANG,114.46,2.896707,215
3,275,LB,82.96,2.319534,297
4,357,OXY,34.22,2.167293,721
5,299,MCHP,76.87,1.657786,321
6,410,SIVB,718.23,1.5712,34
7,23,ALB,246.21,1.522252,100
8,352,OKE,65.50,1.51269,376
9,160,EOG,90.95,1.457531,271


In [10]:
print("""
Building HQM - high quality momentum - Strategy
1-month price returns
3-month price returns
6-month price returns
1-year price returns
""")


Building HQM - high quality momentum - Strategy
1-month price returns
3-month price returns
6-month price returns
1-year price returns



In [11]:
hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]


In [12]:
hqm_dataframe = pd.DataFrame(columns=hqm_columns)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score


In [13]:
 for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   'N/A',
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month6ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month3ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month1ChangePercent'],
                                                   'N/A',
                                                   'N/A'
                                                   ], 
                                                  index = hqm_columns), 
                                        ignore_index = True)

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,159.92,N/A,0.457686,N/A,0.148758,N/A,0.031104,N/A,-0.120153,N/A,N/A
1,AAL,20.66,N/A,0.621137,N/A,-0.096518,N/A,0.009749,N/A,0.013086,N/A,N/A
2,AAP,224.92,N/A,0.419535,N/A,0.13106,N/A,0.052027,N/A,0.048979,N/A,N/A
3,AAPL,152.82,N/A,0.231449,N/A,0.086627,N/A,-0.009144,N/A,-0.008675,N/A,N/A
4,ABBV,111.12,N/A,0.328308,N/A,0.03831,N/A,-0.058765,N/A,0.027654,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,129.75,N/A,0.301954,N/A,0.071468,N/A,0.08585,N/A,-0.014995,N/A,N/A
501,ZBH,146.16,N/A,0.002311,N/A,-0.153011,N/A,-0.045204,N/A,-0.000888,N/A,N/A
502,ZBRA,528.64,N/A,0.786297,N/A,0.004711,N/A,-0.004996,N/A,-0.095318,N/A,N/A
503,ZION,64.25,N/A,1.060243,N/A,0.123624,N/A,0.270069,N/A,0.092745,N/A,N/A


In [14]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]


In [15]:
for row in hqm_dataframe.index:
    for time_period in time_periods:
    
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        if hqm_dataframe.loc[row, change_col] == None:
            hqm_dataframe.loc[row, change_col] = 0.0

for row in hqm_dataframe.index:
    for time_period in time_periods:
        hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row, f'{time_period} Price Return']) / 100

# Print each percentile score to make sure it was calculated properly
for time_period in time_periods:
    print(hqm_dataframe[f'{time_period} Return Percentile'])

#Print the entire DataFrame    
hqm_dataframe

0      0.649505
1      0.758416
2      0.617822
3      0.372277
4      0.506931
         ...   
500    0.461386
501    0.126733
502    0.865347
503    0.924752
504    0.388119
Name: One-Year Return Percentile, Length: 505, dtype: object
0      0.712871
1      0.140594
2      0.679208
3      0.592079
4      0.483168
         ...   
500    0.558416
501    0.077228
502    0.374257
503    0.663366
504    0.843564
Name: Six-Month Return Percentile, Length: 505, dtype: object
0       0.49901
1      0.415842
2       0.59604
3      0.324752
4      0.146535
         ...   
500     0.70297
501     0.19604
502    0.336634
503    0.956436
504    0.437624
Name: Three-Month Return Percentile, Length: 505, dtype: object
0      0.013861
1      0.516832
2      0.685149
3      0.336634
4      0.584158
         ...   
500     0.29505
501    0.390099
502    0.039604
503    0.835644
504    0.538614
Name: One-Month Return Percentile, Length: 505, dtype: object


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,159.92,N/A,0.457686,0.649505,0.148758,0.712871,0.031104,0.49901,-0.120153,0.013861,N/A
1,AAL,20.66,N/A,0.621137,0.758416,-0.096518,0.140594,0.009749,0.415842,0.013086,0.516832,N/A
2,AAP,224.92,N/A,0.419535,0.617822,0.13106,0.679208,0.052027,0.59604,0.048979,0.685149,N/A
3,AAPL,152.82,N/A,0.231449,0.372277,0.086627,0.592079,-0.009144,0.324752,-0.008675,0.336634,N/A
4,ABBV,111.12,N/A,0.328308,0.506931,0.03831,0.483168,-0.058765,0.146535,0.027654,0.584158,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,129.75,N/A,0.301954,0.461386,0.071468,0.558416,0.08585,0.70297,-0.014995,0.29505,N/A
501,ZBH,146.16,N/A,0.002311,0.126733,-0.153011,0.077228,-0.045204,0.19604,-0.000888,0.390099,N/A
502,ZBRA,528.64,N/A,0.786297,0.865347,0.004711,0.374257,-0.004996,0.336634,-0.095318,0.039604,N/A
503,ZION,64.25,N/A,1.060243,0.924752,0.123624,0.663366,0.270069,0.956436,0.092745,0.835644,N/A


In [16]:
for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    print(momentum_percentiles)
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)
    print(hqm_dataframe.loc[row, 'HQM Score'])

[0.6495049504950495, 0.712871287128713, 0.499009900990099, 0.01386138613861386]
0.46881188118811884
[0.7584158415841584, 0.1405940594059406, 0.4158415841584159, 0.5168316831683168]
0.45792079207920794
[0.6178217821782178, 0.6792079207920793, 0.596039603960396, 0.6851485148514851]
0.6445544554455446
[0.3722772277227723, 0.592079207920792, 0.32475247524752476, 0.33663366336633666]
0.40643564356435646
[0.5069306930693069, 0.48316831683168315, 0.14653465346534655, 0.5841584158415841]
0.4301980198019802
[0.36237623762376237, 0.3920792079207921, 0.6396039603960396, 0.22178217821782176]
0.403960396039604
[0.3702970297029703, 0.5504950495049505, 0.7821782178217822, 0.3267326732673267]
0.5074257425742574
[0.23564356435643563, 0.22970297029702968, 0.3900990099009901, 0.09108910891089109]
0.23663366336633662
[0.6851485148514851, 0.8356435643564356, 0.7465346534653466, 0.5623762376237624]
0.7074257425742574
[0.35445544554455444, 0.7603960396039604, 0.40198019801980195, 0.09702970297029703]
0.40346

0.21782178217821782
[0.07326732673267328, 0.22376237623762374, 0.4534653465346535, 0.805940594059406]
0.3891089108910891
[0.24554455445544554, 0.5881188118811881, 0.2930693069306931, 0.05544554455445544]
0.29554455445544553
[0.31485148514851485, 0.04554455445544554, 0.05544554455445544, 0.43366336633663366]
0.21237623762376237
[0.7247524752475247, 0.6772277227722772, 0.40792079207920795, 0.5564356435643565]
0.5915841584158416
[0.08514851485148515, 0.24554455445544554, 0.07326732673267328, 0.4118811881188119]
0.20396039603960398
[0.009900990099009901, 0.09504950495049505, 0.047524752475247525, 0.27524752475247527]
0.10693069306930694
[0.9366336633663366, 0.8297029702970298, 0.9524752475247524, 0.9148514851485149]
0.9084158415841584
[0.34257425742574255, 0.31287128712871287, 0.1425742574257426, 0.14851485148514854]
0.23663366336633662
[0.47722772277227726, 0.45742574257425744, 0.4475247524752475, 0.9168316831683169]
0.5747524752475248
[0.5623762376237624, 0.8237623762376238, 0.8772277227

0.5891089108910892
[0.3564356435643565, 0.5326732673267327, 0.18415841584158416, 0.019801980198019802]
0.2732673267326733
[0.20396039603960398, 0.15643564356435644, 0.3405940594059406, 0.402970297029703]
0.275990099009901
[0.9683168316831683, 0.4178217821782178, 0.8693069306930693, 0.9643564356435644]
0.804950495049505
[0.015841584158415842, 0.031683168316831684, 0.0039603960396039604, 0.0297029702970297]
0.020297029702970298
[0.34851485148514855, 0.499009900990099, 0.2693069306930693, 0.37623762376237624]
0.37326732673267327
[0.7188118811881188, 0.2950495049504951, 0.7643564356435644, 0.3821782178217822]
0.5400990099009901
[0.02772277227722772, 0.04356435643564357, 0.031683168316831684, 0.48316831683168315]
0.14653465346534653
[0.22376237623762374, 0.10891089108910892, 0.30297029702970296, 0.5089108910891089]
0.2861386138613861
[0.900990099009901, 0.7386138613861387, 0.9108910891089108, 0.8495049504950495]
0.85
[0.1188118811881188, 0.3584158415841584, 0.36435643564356435, 0.4029702970

0.2846534653465347
[0.5089108910891089, 0.6574257425742575, 0.5584158415841585, 0.5722772277227722]
0.5742574257425743
[0.8178217821782178, 0.6217821782178218, 0.7623762376237623, 0.7425742574257426]
0.7361386138613861
[0.06930693069306931, 0.3346534653465346, 0.24158415841584158, 0.23168316831683167]
0.21930693069306928
[0.8831683168316832, 0.693069306930693, 0.906930693069307, 0.8831683168316832]
0.8415841584158416
[0.7623762376237623, 0.7564356435643564, 0.7168316831683168, 0.15247524752475247]
0.597029702970297
[0.3405940594059406, 0.15247524752475247, 0.18811881188118812, 0.5128712871287129]
0.2985148514851485
[0.9425742574257425, 0.7544554455445546, 0.6772277227722772, 0.6316831683168317]
0.7514851485148515
[0.7663366336633664, 0.3405940594059406, 0.8198019801980198, 0.021782178217821784]
0.4871287128712871
[0.033663366336633666, 0.29702970297029707, 0.2396039603960396, 0.30495049504950494]
0.2188118811881188
[0.7267326732673267, 0.7267326732673267, 0.6356435643564357, 0.95247524

0.602970297029703
[0.9267326732673267, 0.8554455445544555, 0.9366336633663366, 0.7188118811881188]
0.8594059405940594
[0.6237623762376238, 0.706930693069307, 0.6633663366336634, 0.899009900990099]
0.7232673267326732
[0.9405940594059407, 0.9346534653465346, 0.7881188118811882, 0.2396039603960396]
0.7257425742574257
[0.7405940594059406, 0.9782178217821782, 0.9128712871287129, 0.3782178217821782]
0.7524752475247525
[0.27524752475247527, 0.3445544554455446, 0.5128712871287129, 0.26732673267326734]
0.35000000000000003
[0.45940594059405937, 0.08118811881188119, 0.0297029702970297, 0.04356435643564357]
0.15346534653465346
[0.8415841584158416, 0.2930693069306931, 0.6039603960396039, 0.8851485148514852]
0.655940594059406
[0.8376237623762377, 0.1306930693069307, 0.3267326732673267, 0.7584158415841584]
0.5133663366336634
[0.3465346534653465, 0.5445544554455446, 0.43366336633663366, 0.6]
0.48118811881188117
[0.7029702970297029, 0.8336633663366336, 0.904950495049505, 0.6693069306930692]
0.777722772

0.4386138613861386
[0.5980198019801981, 0.3940594059405941, 0.3782178217821782, 0.36435643564356435]
0.4336633663366337
[0.47128712871287126, 0.6158415841584158, 0.5762376237623762, 0.15643564356435644]
0.4549504950495049
[0.5683168316831683, 0.42772277227722777, 0.8138613861386139, 0.7029702970297029]
0.6282178217821782
[0.5445544554455446, 0.42178217821782177, 0.07920792079207921, 0.13465346534653466]
0.29504950495049503
[0.1405940594059406, 0.23564356435643563, 0.3722772277227723, 0.9128712871287129]
0.4153465346534654
[0.5663366336633664, 0.12673267326732673, 0.16633663366336635, 0.4613861386138614]
0.3301980198019802
[0.48118811881188117, 0.4356435643564357, 0.592079207920792, 0.33069306930693065]
0.45990099009900987
[0.9227722772277228, 0.9148514851485149, 0.7980198019801981, 0.8118811881188118]
0.8618811881188119
[0.4099009900990099, 0.6673267326732674, 0.6297029702970297, 0.7841584158415842]
0.6227722772277228
[0.7346534653465346, 0.2732673267326733, 0.5564356435643565, 0.20198

In [17]:
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,159.92,N/A,0.457686,0.649505,0.148758,0.712871,0.031104,0.49901,-0.120153,0.013861,0.468812
1,AAL,20.66,N/A,0.621137,0.758416,-0.096518,0.140594,0.009749,0.415842,0.013086,0.516832,0.457921
2,AAP,224.92,N/A,0.419535,0.617822,0.13106,0.679208,0.052027,0.59604,0.048979,0.685149,0.644554
3,AAPL,152.82,N/A,0.231449,0.372277,0.086627,0.592079,-0.009144,0.324752,-0.008675,0.336634,0.406436
4,ABBV,111.12,N/A,0.328308,0.506931,0.03831,0.483168,-0.058765,0.146535,0.027654,0.584158,0.430198
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,129.75,N/A,0.301954,0.461386,0.071468,0.558416,0.08585,0.70297,-0.014995,0.29505,0.504455
501,ZBH,146.16,N/A,0.002311,0.126733,-0.153011,0.077228,-0.045204,0.19604,-0.000888,0.390099,0.197525
502,ZBRA,528.64,N/A,0.786297,0.865347,0.004711,0.374257,-0.004996,0.336634,-0.095318,0.039604,0.40396
503,ZION,64.25,N/A,1.060243,0.924752,0.123624,0.663366,0.270069,0.956436,0.092745,0.835644,0.84505


In [18]:
hqm_dataframe.sort_values('HQM Score', ascending=False, inplace=True)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
148,DVN,41.99,N/A,3.628891,1.0,0.892307,1.0,0.576271,0.99802,0.349045,0.99604,0.998515
299,MCHP,75.15,N/A,1.725925,0.990099,0.850017,0.99802,1.185839,1.0,0.835954,1.0,0.99703
317,MRO,17.03,N/A,3.048593,0.99802,0.572326,0.992079,0.383669,0.988119,0.360735,0.99802,0.994059
175,FANG,110.31,N/A,2.852084,0.99604,0.417533,0.982178,0.392756,0.990099,0.318664,0.992079,0.990099
109,COP,77.80,N/A,1.258875,0.960396,0.484609,0.984158,0.358366,0.984158,0.262015,0.986139,0.978713
...,...,...,...,...,...,...,...,...,...,...,...,...
377,PNW,67.49,N/A,-0.130048,0.025743,-0.191415,0.039604,-0.21361,0.009901,-0.08245,0.069307,0.036139
180,FDX,236.68,N/A,-0.190515,0.015842,-0.203817,0.031683,-0.222672,0.00396,-0.102825,0.029703,0.020297
291,LW,57.40,N/A,-0.222331,0.011881,-0.311691,0.007921,-0.273853,0.00198,-0.090312,0.045545,0.016832
240,INCY,66.24,N/A,-0.285819,0.00396,-0.209999,0.029703,-0.16679,0.021782,-0.138618,0.005941,0.015347


In [19]:
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(drop=True, inplace=True)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,DVN,41.990,N/A,3.628891,1.0,0.892307,1.0,0.576271,0.99802,0.349045,0.99604,0.998515
1,MCHP,75.150,N/A,1.725925,0.990099,0.850017,0.99802,1.185839,1.0,0.835954,1.0,0.99703
2,MRO,17.030,N/A,3.048593,0.99802,0.572326,0.992079,0.383669,0.988119,0.360735,0.99802,0.994059
3,FANG,110.310,N/A,2.852084,0.99604,0.417533,0.982178,0.392756,0.990099,0.318664,0.992079,0.990099
4,COP,77.800,N/A,1.258875,0.960396,0.484609,0.984158,0.358366,0.984158,0.262015,0.986139,0.978713
5,PXD,192.320,N/A,1.160881,0.950495,0.308024,0.948515,0.376773,0.986139,0.215022,0.976238,0.965347
6,OKE,67.720,N/A,1.536061,0.986139,0.322488,0.960396,0.248448,0.946535,0.187212,0.966337,0.964851
7,SIVB,722.080,N/A,1.549329,0.988119,0.340764,0.970297,0.237117,0.942574,0.134264,0.936634,0.959406
8,OXY,34.570,N/A,2.184317,0.992079,0.309657,0.950495,0.22392,0.920792,0.175325,0.960396,0.955941
9,MOS,43.940,N/A,1.194692,0.954455,0.249534,0.891089,0.410788,0.994059,0.245143,0.982178,0.955446


In [20]:
portfolio_input()

Enter the size of your portfolio: 1234567


In [21]:
position_size = float(portfolio_size)/len(hqm_dataframe.index)
for row in hqm_dataframe.index:
    hqm_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size/hqm_dataframe.loc[row, 'Price'])
    """ 
    or
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe['Price'][i])
    """
hqm_dataframe

Z:\Projects\strategy_index_screener\.venv\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,DVN,41.990,588,3.628891,1.0,0.892307,1.0,0.576271,0.99802,0.349045,0.99604,0.998515
1,MCHP,75.150,328,1.725925,0.990099,0.850017,0.99802,1.185839,1.0,0.835954,1.0,0.99703
2,MRO,17.030,1449,3.048593,0.99802,0.572326,0.992079,0.383669,0.988119,0.360735,0.99802,0.994059
3,FANG,110.310,223,2.852084,0.99604,0.417533,0.982178,0.392756,0.990099,0.318664,0.992079,0.990099
4,COP,77.800,317,1.258875,0.960396,0.484609,0.984158,0.358366,0.984158,0.262015,0.986139,0.978713
5,PXD,192.320,128,1.160881,0.950495,0.308024,0.948515,0.376773,0.986139,0.215022,0.976238,0.965347
6,OKE,67.720,364,1.536061,0.986139,0.322488,0.960396,0.248448,0.946535,0.187212,0.966337,0.964851
7,SIVB,722.080,34,1.549329,0.988119,0.340764,0.970297,0.237117,0.942574,0.134264,0.936634,0.959406
8,OXY,34.570,714,2.184317,0.992079,0.309657,0.950495,0.22392,0.920792,0.175325,0.960396,0.955941
9,MOS,43.940,561,1.194692,0.954455,0.249534,0.891089,0.410788,0.994059,0.245143,0.982178,0.955446


In [22]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index = False)

background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [23]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', integer_template]
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

In [24]:
writer.save()